In [ ]:
# viz_bot.py
# Retail Data Visualization Bot
#Bu notebook, SQL Server'dan alınan verileri görselleştirip kaydeden bir otomasyon örneğidir.


import os
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from matplotlib.ticker import FuncFormatter
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

# ---------- AYARLAR ----------
mpl.rcParams["font.family"] = "DejaVu Sans"
sns.set_style("whitegrid")
plt.ion()  # interactive mode on (helps some GUIs)

# SQL bağlantısı (kendi bilgini kullan)
server = 'my_server'
database = 'my_database'
conn_str = f"mssql+pyodbc://{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"
engine = create_engine(conn_str, fast_executemany=True)

# PNG klasörü
save_dir = r"C:\data\retail_plots"
os.makedirs(save_dir, exist_ok=True)

# helper: CSV/PNG isimlendir
def safe_filename(name):
    return "".join(c if c.isalnum() or c in ('_', '-') else '_' for c in name)

# helper: kaydetme / gösterme mantığı (robust)
def show_and_ask_save(fig, table_name):
    """
    fig: matplotlib.figure.Figure
    table_name: str
    davranış:
      - eğer headless backend (Agg) ise otomatik kaydet
      - değilse fig'i ekranda göster, kullanıcıdan ok/no ister,
        ok ise fig.savefig ile kaydeder.
    """
    backend = mpl.get_backend().lower()
    filename = os.path.join(save_dir, f"{safe_filename(table_name)}.png")
    # Force a draw
    try:
        fig.canvas.draw()
    except Exception:
        pass

    # Headless (no GUI) backend detection
    if 'agg' in backend:
        # can't show, auto-save
        fig.savefig(filename, bbox_inches="tight", dpi=150)
        print(f"🔒 (Headless) Grafik otomatik kaydedildi: {filename}")
        plt.close(fig)
        return

    # Try to show non-blocking and let GUI render
    try:
        plt.show(block=False)
        # small pause to let GUI event loop draw window
        plt.pause(0.3)
    except Exception:
        # fallback: still attempt to save
        print("⚠️ GUI gösterimde problem var; kaydetmeye çalışıyorum.")
        fig.savefig(filename, bbox_inches="tight", dpi=150)
        print(f"📂 Grafik kaydedildi: {filename}")
        plt.close(fig)
        return

    # ask user
    choice = input(f"💾 {table_name} grafiğini kaydetmek ister misin? (ok/no) >>> ").strip().lower()
    if choice == "ok":
        fig.savefig(filename, bbox_inches="tight", dpi=150)
        print(f"📂 Grafik kaydedildi: {filename}")
    else:
        print("⏭️ Kaydedilmedi.")
    plt.close(fig)


# ---------- PLOT KURALLARI ----------
def plot_table_by_rules(df, table_name):
    """
    df: pandas.DataFrame
    table_name: str (ör. 'tx_by_hour', 'pareto_data' vb.)
    """
    # Normalize column names lower for safe checks
    cols = [c.lower() for c in df.columns]

    # TX BY HOUR
    if table_name == "tx_by_hour" and set(["tx_hour","tx_count"]).issubset(cols):
        fig, ax = plt.subplots(figsize=(10,6))
        sns.barplot(x="tx_hour", y="tx_count", data=df.sort_values("tx_hour"), ax=ax, color="darkorange")
        ax.set_title("Saatlere Göre Transaction Dağılımı")
        ax.set_xlabel("Hour")
        ax.set_ylabel("Transaction Count")
        ax.set_xticks(range(0,24))
        show_and_ask_save(fig, table_name)
        return

    # TX BY ITEM (Top10)
    if table_name == "tx_by_item" and set(["itemid","tx_count"]).issubset(cols):
        top10 = df.sort_values("tx_count", ascending=False).head(10)
        fig, ax = plt.subplots(figsize=(10,6))
        sns.barplot(x=top10["itemid"].astype(str), y=top10["tx_count"], ax=ax, color="royalblue")
        ax.set_title("En Çok Satılan 10 Ürün")
        ax.set_xlabel("Item ID")
        ax.set_ylabel("Tx Count")
        ax.set_xticklabels(top10["itemid"].astype(str), rotation=30, ha="right")
        show_and_ask_save(fig, table_name)
        return

    # BESTSELLER ITEMS (assume already top10)
    if table_name == "bestseller_items" and set(["itemid","tx_count"]).issubset(cols):
        fig, ax = plt.subplots(figsize=(10,6))
        sns.barplot(x=df["itemid"].astype(str), y=df["tx_count"], ax=ax, color="seagreen")
        ax.set_title("Bestseller Ürünler (Top 10)")
        ax.set_xticklabels(df["itemid"].astype(str), rotation=30, ha="right")
        show_and_ask_save(fig, table_name)
        return

    # TX_DISTRIBUTION (categorical order)
    if table_name == "tx_distribution" and "sales_range" in cols:
        # Ensure categorical ordering
        order = ["1", "2-5", "6-10", "11-20", "21-50", "51-100", "100+"]
        # If df has product_count or total_tx choose product_count for display
        y_col = "product_count" if "product_count" in cols else ("total_tx" if "total_tx" in cols else df.columns[1])
        # Convert to categorical to preserve order in plot
        df_plot = df.copy()
        df_plot['sales_range'] = pd.Categorical(df_plot['sales_range'], categories=order, ordered=True)
        df_plot = df_plot.sort_values('sales_range')
        fig, ax = plt.subplots(figsize=(8,4))
        sns.barplot(x="sales_range", y=y_col, data=df_plot, order=order, ax=ax, color="purple")
        ax.set_title("Satış Dağılımı (Gruplar)")
        ax.set_xlabel("Sales Range")
        ax.set_ylabel(y_col)
        show_and_ask_save(fig, table_name)
        return

    # PARETO_DATA: iki senaryo
    # A) Grup bazlı pareto: columns include sales_range, group_tx, cum_pct
    if table_name == "pareto_data" and set(["sales_range","group_tx","cum_pct"]).issubset(cols):
        order = ["1", "2-5", "6-10", "11-20", "21-50", "51-100", "100+"]
        df_plot = df.copy()
        df_plot['sales_range'] = pd.Categorical(df_plot['sales_range'], categories=order, ordered=True)
        df_plot = df_plot.sort_values('sales_range')

        fig, ax1 = plt.subplots(figsize=(10,6))
        ax2 = ax1.twinx()
        sns.barplot(x="sales_range", y="group_tx", data=df_plot, ax=ax1, order=order, color="skyblue")
        sns.lineplot(x="sales_range", y="cum_pct", data=df_plot, ax=ax2, marker="o", color="red")
        ax1.set_xlabel("Sales Range")
        ax1.set_ylabel("Group Tx (Toplam Satış)")
        ax2.set_ylabel("Kümülatif %")
        ax2.set_ylim(0,100)
        ax1.set_title("Pareto Analizi (Gruplu)")
        # inset: küçük tablo/plot sağ üst
        axins = inset_axes(ax1, width="35%", height="35%", loc="upper right")
        sns.barplot(x="sales_range", y="group_tx", data=df_plot, ax=axins, order=order, color="purple")
        axins.set_xticklabels([])
        axins.set_yticklabels([])
        axins.set_title("Grup Toplamları", fontsize=8)

        show_and_ask_save(fig, table_name + "_groups")
        return

    # PARETO_DATA: ürün bazlı (itemid, tx_count, cum_pct)
    if table_name == "pareto_data" and set(["itemid","tx_count","cum_pct"]).issubset(cols):
        df_top = df.sort_values("tx_count", ascending=False).head(20)
        fig, ax1 = plt.subplots(figsize=(12,6))
        ax2 = ax1.twinx()
        sns.barplot(x=df_top["itemid"].astype(str), y=df_top["tx_count"], ax=ax1, color="skyblue")
        sns.lineplot(x=df_top["itemid"].astype(str), y=df_top["cum_pct"], ax=ax2, marker="o", color="red")
        ax1.set_xticklabels(df_top["itemid"].astype(str), rotation=45, ha="right")
        ax1.set_ylabel("Tx Count")
        ax2.set_ylabel("Kümülatif %")
        ax2.set_ylim(0,100)
        ax1.set_title("Pareto Analizi (İlk 20 Ürün)")
        show_and_ask_save(fig, table_name + "_top20")
        return

    # TOTAL_TX
    if table_name == "total_tx" and "total_transactions" in cols:
        fig, ax = plt.subplots(figsize=(6,4))
        total = int(df["total_transactions"].iloc[0])
        ax.text(0.5, 0.5, f"Toplam Transaction: {total:,}", fontsize=20, ha="center", va="center")
        ax.axis('off')
        show_and_ask_save(fig, table_name)
        return

    # EVENT_TYPES
    if table_name == "event_types" and set(["event","event_count"]).issubset(cols):
        fig, ax = plt.subplots(figsize=(10,6))
        sns.barplot(x="event", y="event_count", data=df, ax=ax, color="royalblue")
        ax.set_title("Event Türleri")
        ax.set_xlabel("Event")
        ax.set_ylabel("Event Count")
        # if big numbers, format with commas
        ax.yaxis.set_major_formatter(FuncFormatter(lambda x, p: format(int(x), ',')))
        ax.set_xticklabels(ax.get_xticklabels(), rotation=30, ha="right")
        show_and_ask_save(fig, table_name)
        return

    # HOURLY_EVENTS
    if table_name == "hourly_events" and set(["event_hour","events_per_hour"]).issubset(cols):
        fig, ax = plt.subplots(figsize=(10,6))
        sns.barplot(x="event_hour", y="events_per_hour", data=df.sort_values("event_hour"), ax=ax, color="darkcyan")
        ax.set_title("Saatlik Event Dağılımı")
        ax.set_xlabel("Hour")
        ax.set_ylabel("Events per hour")
        ax.set_xticks(range(0,24))
        show_and_ask_save(fig, table_name)
        return

    # FUNNEL_STRICT_SUMMARY
    if table_name == "FUNNEL_STRICT_SUMMARY":
        # Expecting columns like step1_viewers, passed_view_to_add, passed_add_to_tx, pct_...
        # Build a small df for steps
        steps = []
        if "step1_viewers" in cols:
            steps.append(("Viewers", int(df["step1_viewers"].iloc[0])))
        if "passed_view_to_add" in cols:
            steps.append(("Passed View→Add", int(df["passed_view_to_add"].iloc[0])))
        if "passed_add_to_tx" in cols:
            steps.append(("Passed Add→Tx", int(df["passed_add_to_tx"].iloc[0])))

        if not steps:
            print("ℹ️ Funnel için beklenen step kolonları yok.")
            return

        funnel_df = pd.DataFrame(steps, columns=["step","user_count"])
        fig, ax = plt.subplots(figsize=(8,5))
        sns.barplot(x="step", y="user_count", data=funnel_df, ax=ax, color="crimson")
        ax.set_title("Funnel Analizi (View→Add→Tx)")
        ax.set_yscale("log")  # log scale as requested
        ax.set_ylabel("User Count (log scale)")
        show_and_ask_save(fig, table_name)
        return

    # FALLBACK: sayısal kolonlardan histogram / preview
    numeric_cols = df.select_dtypes(include="number").columns.tolist()
    fig, ax = plt.subplots(figsize=(10,6))
    if len(numeric_cols) >= 1:
        sns.histplot(df[numeric_cols[0]].dropna(), bins=30, kde=False, ax=ax, color="gray")
        ax.set_title(f"Histogram: {numeric_cols[0]}")
        show_and_ask_save(fig, table_name + "_hist")
        return
    else:
        # Table preview
        ax.axis('off')
        tbl = ax.table(cellText=df.head(10).values, colLabels=df.columns, loc='center')
        tbl.auto_set_font_size(False)
        tbl.set_fontsize(8)
        tbl.scale(1,1.5)
        ax.set_title("Table preview (first 10 rows)")
        show_and_ask_save(fig, table_name + "_preview")
        return

# ---------- MAIN: tabloları sırayla işle ----------
tables = [
    "tx_by_hour",
    "tx_by_item",
    "bestseller_items",
    "tx_distribution",
    "pareto_data",
    "total_tx",
    "event_types",
    "hourly_events",
    "FUNNEL_STRICT_SUMMARY"
]

for table in tables:
    try:
        df = pd.read_sql(f"SELECT * FROM dbo.{table}", engine)
        print(f"\n✅ Tablo çekildi: {table} — satır: {len(df)}")
        # kısa özet
        print("Kolonlar:", list(df.columns))
        if not df.empty:
            # min-max (sayısal)
            nums = df.select_dtypes(include="number")
            if not nums.empty:
                print("Min-Max (sayısal):")
                print(nums.agg(['min','max']).T)
            print("İlk 5 satır:")
            print(df.head())
        plot_table_by_rules(df, table)
    except Exception as exc:
        print(f"⚠️ Hata tablo {table} çekilirken/işlenirken: {exc}")

print("\n✅ İşlem tamam. PNG'ler klasörde:", save_dir)
